In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json

### Collect all dcm related information together

In [105]:
# load file infos
dcm_info = pd.read_csv("all_prediction/dcm_file_info.csv")
p_info = pd.read_csv("all_prediction/Patient_EID.csv", sep = '\t')
filename2id_82 = pd.read_csv("all_prediction/out_82/filename2id_82.csv", index_col = 0)
filename2id_93 = pd.read_csv("all_prediction/out_93/filename2id_93.csv", index_col = 0)

In [106]:
dcm_info = p_info.merge(dcm_info, left_on = 'UID', right_on = 'file_name', how = 'inner').drop(columns = ['UID'])
dcm_info['file_name'] = dcm_info['file_name'].apply(lambda x: x[:-4])

In [107]:
filename2id_82['img_size'] = "816x288"
filename2id_93['img_size'] = "960x384"

In [108]:
filename2id = pd.concat([filename2id_82, filename2id_93])
filename2id['old_name'] = filename2id['old_name'].apply(lambda x: x[:-4])

In [109]:
img_info = dcm_info.merge(filename2id, left_on = "file_name", right_on = "old_name", how = "inner").drop(columns = ['old_name']).rename(columns = {'newName': 'image_id'})

In [111]:
img_info['image_id'] = img_info['image_id'].apply(lambda x: x[:-4])
img_info['p_age'] = img_info['p_age'].apply(lambda x: float(x.replace('Y', '')))

In [113]:
img_info.to_csv("all_prediction/all_dcm_info.csv", index = False)

In [2]:
all_dcm_info = pd.read_csv("all_prediction/all_dcm_info.csv"); all_dcm_info.head()

,Patient EID,file_name,p_id,p_sex,p_age,p_weight,p_height,p_ethnicity,image_id,img_size
0,1000276,1.2.840.113619.2.110.210419.20160630134650.1.1...,BJ MU YD 2B,F,67.0,60.0,1.59,White,8760,960x384
1,1000308,1.2.840.113619.2.110.212038.20190713145954.1.1...,MM QC ST M2,M,66.0,90.0,1.82,White,15998,816x288
2,1000320,1.2.840.113619.2.110.210419.20180924120844.1.1...,2A UV ZZ V9,F,64.0,79.0,1.66,White,2879,816x288
3,1000742,1.2.840.113619.2.110.210419.20170305174208.1.1...,BB QB 3J 4C,F,67.0,77.0,1.61,White,12347,960x384
4,1000951,1.2.840.113619.2.110.212038.20190307144208.1.1...,PE 9K HF 5V,M,78.0,60.0,1.70,White,14366,816x288


### Load all prediction results and convert to json

In [3]:
pred_82_hrnet32 = json.load(open("all_prediction/hrnet32_82_pred.json"))
pred_93_hrnet32 = json.load(open("all_prediction/hrnet32_93_pred.json"))

In [116]:
pred_82 = json.load(open("all_prediction/out_82/w48_384x288_adam_lr1e-3/results/keypoints_DXA_images_results_0.json"))
pred_93 = json.load(open("all_prediction/out_93/w48_384x288_adam_lr1e-3/results/keypoints_DXA_images_results_0.json"))

In [6]:
pred_82_hrnet32['00007']

[[91.80082702636719, 104.92684936523438],
 [158.67330932617188, 102.52780151367188],
 [68.57380676269531, 122.09596252441406],
 [183.84519958496094, 119.68023681640625],
 [84.67841339111328, 155.212158203125],
 [171.88943481445312, 152.32269287109375],
 [112.96192932128906, 192.94151306152344],
 [146.41798400878906, 191.5376434326172],
 [130.135498046875, 179.15628051757812],
 [102.7840805053711, 147.3951416015625],
 [152.47885131835938, 147.0712127685547],
 [125.96241760253906, 131.56491088867188],
 [129.25845336914062, 167.92706298828125],
 [70.55278778076172, 164.2945098876953],
 [188.2298126220703, 162.6647186279297],
 [113.80331420898438, 181.8066864013672],
 [144.6820068359375, 180.0986785888672]]

In [7]:
len(pred_82_hrnet32.keys())

21981

In [8]:
len(pred_93_hrnet32.keys())

17488

### from `deep-high-resolution-net.pytorch`

In [144]:
all_pred = {'816x288': [], '960x384': []}

for i in range(len(pred_82)):
    img_id = "{:05d}".format(pred_82[i]['image_id'])
    single_img = {img_id: {}} # image_id

    kps = np.array(pred_82[i]['keypoints']).reshape(-1, 3)[:, :2]

    single_img[img_id]['iliac_crest_left'] = kps[0].tolist()
    single_img[img_id]['iliac_crest_right'] = kps[1].tolist()
    single_img[img_id]['iliac_spine_left'] = kps[2].tolist()
    single_img[img_id]['iliac_spine_right'] = kps[3].tolist()
    single_img[img_id]['iliopubic_eminence_left'] = kps[4].tolist()
    single_img[img_id]['iliopubic_eminence_right'] = kps[5].tolist()
    single_img[img_id]['inferior_pubic_ramus_left'] = kps[6].tolist()
    single_img[img_id]['inferior_pubic_ramus_right'] = kps[7].tolist()
    single_img[img_id]['pubic_arch'] = kps[8].tolist()
    single_img[img_id]['sciatic_notch_left'] = kps[9].tolist()
    single_img[img_id]['sciatic_notch_right'] = kps[10].tolist()
    single_img[img_id]['sacrum'] = kps[11].tolist()
    single_img[img_id]['pubic_tubercle'] = kps[12].tolist()
    single_img[img_id]['trochanter_left'] = kps[13].tolist()
    single_img[img_id]['trochanter_right'] = kps[14].tolist()
    single_img[img_id]['obturator_left'] = kps[15].tolist()
    single_img[img_id]['obturator_right'] = kps[16].tolist()

    all_pred['816x288'].append(single_img)

for i in range(len(pred_93)):
    img_id = "{:05d}".format(pred_93[i]['image_id'])
    single_img = {img_id: {}} # image_id

    kps = np.array(pred_93[i]['keypoints']).reshape(-1, 3)[:, :2]

    single_img[img_id]['iliac_crest_left'] = kps[0].tolist()
    single_img[img_id]['iliac_crest_right'] = kps[1].tolist()
    single_img[img_id]['iliac_spine_left'] = kps[2].tolist()
    single_img[img_id]['iliac_spine_right'] = kps[3].tolist()
    single_img[img_id]['iliopubic_eminence_left'] = kps[4].tolist()
    single_img[img_id]['iliopubic_eminence_right'] = kps[5].tolist()
    single_img[img_id]['inferior_pubic_ramus_left'] = kps[6].tolist()
    single_img[img_id]['inferior_pubic_ramus_right'] = kps[7].tolist()
    single_img[img_id]['pubic_arch'] = kps[8].tolist()
    single_img[img_id]['sciatic_notch_left'] = kps[9].tolist()
    single_img[img_id]['sciatic_notch_right'] = kps[10].tolist()
    single_img[img_id]['sacrum'] = kps[11].tolist()
    single_img[img_id]['pubic_tubercle'] = kps[12].tolist()
    single_img[img_id]['trochanter_left'] = kps[13].tolist()
    single_img[img_id]['trochanter_right'] = kps[14].tolist()
    single_img[img_id]['obturator_left'] = kps[15].tolist()
    single_img[img_id]['obturator_right'] = kps[16].tolist()

    all_pred['960x384'].append(single_img)

In [145]:
with open("./all_prediction/kps_coordinate.json", "w") as OUTPUT:
    json.dump(all_pred, OUTPUT, indent="  ")

In [24]:
kps = json.load(open("./all_prediction/kps_coordinate.json"))

In [25]:
kps

{'816x288': [{'00001': {'iliac_crest_left': [172.6829376220703,
     314.6451721191406],
    'iliac_crest_right': [183.294921875, 325.2571716308594],
    'iliac_spine_left': [98.39908599853516, 325.2571716308594],
    'iliac_spine_right': [188.6009063720703, 357.0931091308594],
    'iliopubic_eminence_left': [98.39908599853516, 346.4811096191406],
    'iliopubic_eminence_right': [183.294921875, 383.623046875],
    'inferior_pubic_ramus_left': [103.705078125, 388.9290466308594],
    'inferior_pubic_ramus_right': [151.458984375, 404.8470153808594],
    'pubic_arch': [130.2350311279297, 404.8470153808594],
    'sciatic_notch_left': [119.623046875, 357.0931091308594],
    'sciatic_notch_right': [167.376953125, 362.3990783691406],
    'sacrum': [146.1529998779297, 367.705078125],
    'pubic_tubercle': [140.8470001220703, 367.705078125],
    'trochanter_left': [87.787109375, 373.0110778808594],
    'trochanter_right': [199.212890625, 378.3170471191406],
    'obturator_left': [124.92903900146

### from hrnet32

In [31]:
all_pred = {'816x288': [], '960x384': []}

for img_id in sorted(pred_82_hrnet32.keys()):
    single_img = {img_id: {}}
    kps = pred_82_hrnet32[img_id]
    single_img[img_id]['iliac_crest_left'] = kps[0]
    single_img[img_id]['iliac_crest_right'] = kps[1]
    single_img[img_id]['iliac_spine_left'] = kps[2]
    single_img[img_id]['iliac_spine_right'] = kps[3]
    single_img[img_id]['iliopubic_eminence_left'] = kps[4]
    single_img[img_id]['iliopubic_eminence_right'] = kps[5]
    single_img[img_id]['inferior_pubic_ramus_left'] = kps[6]
    single_img[img_id]['inferior_pubic_ramus_right'] = kps[7]
    single_img[img_id]['pubic_arch'] = kps[8]
    single_img[img_id]['sciatic_notch_left'] = kps[9]
    single_img[img_id]['sciatic_notch_right'] = kps[10]
    single_img[img_id]['sacrum'] = kps[11]
    single_img[img_id]['pubic_tubercle'] = kps[12]
    single_img[img_id]['trochanter_left'] = kps[13]
    single_img[img_id]['trochanter_right'] = kps[14]
    single_img[img_id]['obturator_left'] = kps[15]
    single_img[img_id]['obturator_right'] = kps[16]
    all_pred['816x288'].append(single_img)

for img_id in sorted(pred_93_hrnet32.keys()):
    single_img = {img_id: {}}
    kps = pred_93_hrnet32[img_id]
    single_img[img_id]['iliac_crest_left'] = kps[0]
    single_img[img_id]['iliac_crest_right'] = kps[1]
    single_img[img_id]['iliac_spine_left'] = kps[2]
    single_img[img_id]['iliac_spine_right'] = kps[3]
    single_img[img_id]['iliopubic_eminence_left'] = kps[4]
    single_img[img_id]['iliopubic_eminence_right'] = kps[5]
    single_img[img_id]['inferior_pubic_ramus_left'] = kps[6]
    single_img[img_id]['inferior_pubic_ramus_right'] = kps[7]
    single_img[img_id]['pubic_arch'] = kps[8]
    single_img[img_id]['sciatic_notch_left'] = kps[9]
    single_img[img_id]['sciatic_notch_right'] = kps[10]
    single_img[img_id]['sacrum'] = kps[11]
    single_img[img_id]['pubic_tubercle'] = kps[12]
    single_img[img_id]['trochanter_left'] = kps[13]
    single_img[img_id]['trochanter_right'] = kps[14]
    single_img[img_id]['obturator_left'] = kps[15]
    single_img[img_id]['obturator_right'] = kps[16]
    all_pred['960x384'].append(single_img)

In [32]:
with open("./all_prediction/kps_coordinate_hrnet32.json", "w") as f:
    json.dump(all_pred, f)